# 初期化

In [ ]:
!pip install fastlabel

# SDK  Example codes


## 1. データセットの作成とモデル学習の実行

#### 使い方
1. 名前が"fastlabel-sample"のデータセットが既にある場合、削除する
2. パラメータのFASTLABEL_ACCESS_TOKENを設定
3. 実行

#### パラメータ
1. FASTLABEL_ACCESS_TOKEN:　プラットフォーム上で発行するアクセストークン

In [ ]:
import fastlabel
import json
import os

FASTLABEL_ACCESS_TOKEN = "sample-token" # Replace with your access token
DATASET_NAME = "fastlabel-sample" # Don't need to change this

# Initialize client
os.environ["FASTLABEL_ACCESS_TOKEN"] = FASTLABEL_ACCESS_TOKEN
client = fastlabel.Client()

# create dataset
dataset = client.create_dataset(
    name=DATASET_NAME,
)

# load annotations
def remove_unnecessary_keys(annotations):
    keys_to_remove = ["id", "color"]
    clean_annotation = [
        {k: v for k, v in annotation.items() if k not in keys_to_remove}
        for annotation in annotations
    ]
    return clean_annotation


with open("./sample_data/annotation/fastlabel/annotations.json", "r") as f:
    annotations = json.load(f)
    name_annotation_map = {}
    for annotation in annotations:
        name = annotation["name"]
        name_annotation_map[name] = remove_unnecessary_keys(annotation["annotations"])

# create dataset objects
objects = [
    {"name": "sample1.jpg", "file_path": "./sample_data/images/sample1.jpg"},
    {"name": "sample2.jpg", "file_path": "./sample_data/images/sample2.jpg"},
]

for object in objects:
    dataset_object = client.create_dataset_object(
        dataset=DATASET_NAME,
        name=object["name"],
        file_path=object["file_path"],
        tags=[],  # max 5 tags per dataset object.
        licenses=["MIT"],  # max 10 licenses per dataset object
        annotations=name_annotation_map[object["name"]],
    )

# train model
training_job = client.execute_training_job(
    dataset_name=DATASET_NAME,
    base_model_name="fastlabel_object_detection_light",
    epoch=300,
)


## 2. データセットの全てのオブジェクトを取得

#### 使い方
1. オブジェクトが登録されたデータセットを用意する
2. パラメータのFASTLABEL_ACCESS_TOKENを設定
3. パラメータのDATASET_NAMEに用意したプロジェクトの名前を設定
4. 実行

#### パラメータ
1. FASTLABEL_ACCESS_TOKEN:　プラットフォーム上で発行するアクセストークン
2. DATASET_NAME:　データセットの名前

In [ ]:
import fastlabel
import os

FASTLABEL_ACCESS_TOKEN = "sample-token" # Replace with your access token
DATASET_NAME = "fastlabel-sample" # Replace with your dataset name

# Initialize client
os.environ["FASTLABEL_ACCESS_TOKEN"] = FASTLABEL_ACCESS_TOKEN
client = fastlabel.Client()


def get_all_dataset_objects(dataset_name: str, tags: list[str] = None):
    # Iterate pages until new tasks are empty.
    result = []
    offset = None

    print("Fetching dataset objects...")
    while True:
        datasets = client.get_dataset_objects(
            dataset=dataset_name, tags=tags, offset=offset
        )
        result.extend(datasets)

        if len(datasets) > 0:
            offset = len(result)
            print(offset, " fetched")
        else:
            break

    print("Fetching dataset objects finished!")
    print("Total dataset object count: ", len(result))
    return result


if __name__ == "__main__":
    print(get_all_dataset_objects(dataset_name=DATASET_NAME))
